In [1]:
import os
import sys

home=os.path.expanduser("~")
os.environ["SPARK_HOME"]=os.path.join(home, 'spark-2.0.0-bin-hadoop2.7')
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

## 텍스트 데이터, 웹에서 발생, 해야할 작업들이 많다.
* Extract Transform Load: ETL

* Vector => numpy vector와 같은 기능을 한다. dense와 sparse vector로 구분한다.
* Labeled Point => 분류를 의미하는 (클래스 또는 label)과(이진 분류). 속성 features(결정자) 이 묶인 구조로서, supervised learning에 사용된다. 목표가 연속적일 때는 다르게 써야한다. 클래스와 label은 맞다 아니다만 구분
* Matrix => numpy matrix와 같은 특징을 가진다.

* mllib => RDD
* ml => dataframe

# denseVec

In [11]:
import numpy as np

dv = np.array([1.0, 2.1, 3])

print dv, type(dv)

[1.  2.1 3. ] <type 'numpy.ndarray'>


In [8]:
from pyspark.mllib.linalg import Vectors

dv1mllib=Vectors.dense([1.0, 2.1, 3])
print dv1mllib, type(dv1mllib)


[1.0,2.1,3.0] <class 'pyspark.mllib.linalg.DenseVector'>


In [9]:
from pyspark.ml.linalg import Vectors

dv1ml=Vectors.dense([1.0, 2.1, 3])
print dv1ml, type(dv1ml)

[1.0,2.1,3.0] <class 'pyspark.ml.linalg.DenseVector'>


In [10]:
for e in dv1ml:
    print e,

1.0 2.1 3.0


In [15]:
print dv1ml.dot(dv1ml) #pyspark => 분산

14.41


In [16]:
np.dot(dv,dv) #numpy => 로컬

14.41

# SparseVec

In [17]:
sv1 = Vectors.sparse(3, [1, 2], [1.0, 3.0])
# toArray -> Spase -> Dense
print sv1.toArray()

[0. 1. 3.]


In [58]:
import numpy as np
import scipy.sparse as sps

row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6]) # (0, 0, 1) (0, 2, 2) (1, 2, 3) (2, 0, 4) (2, 1, 5) (2, 2, 6)
mtx = sps.csc_matrix((data, (row, col)), shape=(3, 3))
print mtx.todense()   

[[1 0 2]
 [0 0 3]
 [4 5 6]]


# RowMatrix

In [21]:
p = [[1.0,2.0,3.0],[1.1,2.1,3.1],[1.2,2.2,3.3]]
my=spark.sparkContext.parallelize(p)
my.collect()

[[1.0, 2.0, 3.0], [1.1, 2.1, 3.1], [1.2, 2.2, 3.3]]

In [23]:
from pyspark.mllib.linalg.distributed import RowMatrix
rm=RowMatrix(my)
rm.rows.collect()

[DenseVector([1.0, 2.0, 3.0]),
 DenseVector([1.1, 2.1, 3.1]),
 DenseVector([1.2, 2.2, 3.3])]

# Labled Point

In [24]:
from pyspark.mllib.regression import LabeledPoint

# (성공, 실패), (features)
print LabeledPoint(1.0, [1.0, 2.0, 3.0])

(1.0,[1.0,2.0,3.0])


In [25]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

# (연속값 분류), (features)
print LabeledPoint(1992, Vectors.sparse(10, {0: 3.0, 1:5.5, 2: 10.0}))

(1992.0,(10,[0,1,2],[3.0,5.5,10.0]))


In [26]:
from pyspark.mllib.regression import LabeledPoint

LabeledPoint(1.0, dv1mllib)

LabeledPoint(1.0, [1.0,2.1,3.0])

In [28]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

# dvm1은 dataframe을 사용중이다. 변환을 해ㅏ고 사용해야한다.

LabeledPoint(1.0, Vectors.fromML(dv1ml))

LabeledPoint(1.0, [1.0,2.1,3.0])

In [30]:
from pyspark.mllib.regression import LabeledPoint
p = [[1,[1.0,2.0,3.0]],[1,[1.1,2.1,3.1]],[0,[1.2,2.2,3.3]]]
trainDf=spark.createDataFrame(p)
trainDf.collect()

[Row(_1=1, _2=[1.0, 2.0, 3.0]),
 Row(_1=1, _2=[1.1, 2.1, 3.1]),
 Row(_1=0, _2=[1.2, 2.2, 3.3])]

In [31]:
from pyspark.mllib.linalg import Vectors

trainDf = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, 1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, 0.5]))], ["label", "features"])
trainDf.collect()

[Row(label=1.0, features=DenseVector([0.0, 1.1, 0.1])),
 Row(label=0.0, features=DenseVector([2.0, 1.0, 1.0])),
 Row(label=0.0, features=DenseVector([2.0, 1.3, 1.0])),
 Row(label=1.0, features=DenseVector([0.0, 1.2, 0.5]))]

In [34]:
from pyspark.mllib.linalg import SparseVector, VectorUDT
from pyspark.sql.types import StructType, StructField, DoubleType
_rdd = spark.sparkContext.parallelize([
    (0.0, SparseVector(4, {1: 1.0, 3: 5.5})),
    (1.0, SparseVector(4, {0: -1.0, 2: 0.5}))])

schema = StructType([
    StructField("label", DoubleType(), True),
    StructField("features", VectorUDT(), True)
])
trainDf=_rdd.toDF(schema)
trainDf.printSchema()
trainDf.collect()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



[Row(label=0.0, features=SparseVector(4, {1: 1.0, 3: 5.5})),
 Row(label=1.0, features=SparseVector(4, {0: -1.0, 2: 0.5}))]

In [37]:
from pyspark.sql.functions import udf
#from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.mllib.linalg import DenseVector, VectorUDT
#myudf=udf(lambda x: Vectors.dense(x), VectorUDT())
#myudf=udf(lambda x: Vectors.dense(x))
myudf=udf(lambda x: DenseVector(x.toArray()), VectorUDT())
_trainDf2=trainDf.withColumn('dvf',myudf(trainDf.features))
_trainDf2.printSchema()
_trainDf2.show()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- dvf: vector (nullable = true)

+-----+--------------------+------------------+
|label|            features|               dvf|
+-----+--------------------+------------------+
|  0.0| (4,[1,3],[1.0,5.5])| [0.0,1.0,0.0,5.5]|
|  1.0|(4,[0,2],[-1.0,0.5])|[-1.0,0.0,0.5,0.0]|
+-----+--------------------+------------------+



# maxtrix

In [38]:
from pyspark.mllib.linalg import Matrix, Matrices

# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
dm = Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])

# Create a sparse matrix ((9.0, 0.0), (0.0, 8.0), (0.0, 6.0))
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])

# RDD 훈련데이터 만들기

In [93]:
filePath=os.path.join(os.environ["SPARK_HOME"],'data','mllib','sample_svm_data.txt')
fileRdd=spark.sparkContext.textFile(filePath).map(lambda line: [float(x) for x in line.split(' ')])

    
from pyspark.mllib.regression import LabeledPoint
trainRdd=fileRdd.map(lambda line : LabeledPoint(line[0], line[1:]))

cnt = 1
for line in trainRdd.collect():
    print "[",cnt,"]",""
    print "[LABLE:",line.label,"]"
    for elem in line.features:
        print elem,", ",
    cnt+=1
    print '\n'

[ 1 ] 
[LABLE: 1.0 ]
0.0 ,  2.52078447201548 ,  0.0 ,  0.0 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 2 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 3 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  2.061393766919624 ,  0.0 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 4 ] 
[LABLE: 1.0 ]
0.0 ,  0.0 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 5 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  0.0 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.05500287586

[LABLE: 0.0 ]
2.857738033247042 ,  2.52078447201548 ,  0.0 ,  2.619965104088255 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 57 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 58 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 59 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  0.0 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  0.0 ,  0.0 ,  2.122974378789621 ,  0.0 ,  0.0 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 60 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  2.52078447201548 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  0.0 ,  0.0

2.857738033247042 ,  0.0 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  2.122974378789621 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 105 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  4.745052855503306 ,  0.0 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 106 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 107 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 108 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  2.122974378789621 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  2.055002875

0.0 ,  2.52078447201548 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 155 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 156 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  0.0 ,  2.061393766919624 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.122974378789621 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 157 ] 
[LABLE: 1.0 ]
0.0 ,  0.0 ,  2.061393766919624 ,  2.619965104088255 ,  4.745052855503306 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 158 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  2.52078447201548 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 , 

2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  0.0 ,  2.000347299268466 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 204 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  2.122974378789621 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 205 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  2.122974378789621 ,  0.0 ,  0.0 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 206 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  0.0 ,  2.000347299268466 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 207 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  2.52078447201548 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  0.0 ,  2.000347299268466 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,

2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  2.122974378789621 ,  0.0 ,  0.0 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 255 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 256 ] 
[LABLE: 1.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 257 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 258 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  2.061393766919624 ,  0.0 ,  4.745052855503306 ,  0.0 ,  0.0 ,  2.122974378789621 ,  2.228387042742021 ,  2.228387042

2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 305 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  2.000347299268466 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 306 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  2.52078447201548 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  0.0 ,  2.000347299268466 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 307 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  0.0 ,  2.619965104088255 ,  0.0 ,  0.0 ,  0.0 ,  2.122974378789621 ,  2.228387042742021 ,  2.228387042742023 ,  0.0 ,  2.055002875864414 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  

[ 308 ] 
[LABLE: 0.0 ]
2.857738033247042 ,  0.0 ,  2.061393766919624 ,  2.619965104088255 ,  0.0 ,  2.004684436494304 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.0550028758

# svm

In [92]:
fsvm=os.path.join(os.environ["SPARK_HOME"],'data','mllib','sample_libsvm_data.txt')
dfsvm = spark.read.format("libsvm").load(fsvm)
dfsvm.printSchema()

cnt = 1;
for line in dfsvm.collect():
    print "[",cnt,"]",""
    print "[LABLE:",line.label,"]",
    for elem in line.features.toArray():
        print elem, ",",
    print "\n"
    cnt+=1

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

[ 1 ] 
[LABLE: 0.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 51.0 , 159.0 , 253.0 , 159.0 , 50.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 

[LABLE: 1.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 145.0 , 255.0 , 211.0 , 31.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 32.0 , 237.0 , 253.0 , 252.0 , 71.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 ,

[LABLE: 1.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 1.0 , 168.0 , 242.0 , 28.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0

[LABLE: 1.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 121.0 , 254.0 , 136.0 , 0.0 , 0.0 , 0.

[LABLE: 0.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 73.0 , 253.0 , 227.0 , 73.0 , 21.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 73.0 , 251.0 , 251.0 , 251.0 , 174.0 , 0.0 , 0.0 

...

[LABLE: 1.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 42.0 , 232.0 , 254.0 , 58.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 86.0 , 253.0 , 253.0 , 58.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.

[LABLE: 1.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 11.0 , 150.0 , 72.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 37.0 , 251.0 , 71.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 63.0 , 251.0 , 71.0 , 0.0 , 0.0 , 0.0 , 0.

[LABLE: 0.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 45.0 , 254.0 , 254.0 , 254.0 , 148.0 , 24.0 , 9.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 43.0 , 254.0 , 252.0 , 252.0 , 252.0 , 252.0 , 252.0 , 121

[LABLE: 0.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 60.0 , 96.0 , 96.0 , 48.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 16.0 , 171.0 , 228.0 , 253.0 , 251.0 , 220.0 , 51.0 , 32.0 , 0.0 , 0.0

[LABLE: 1.0 ] 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 218.0 , 170.0 , 108.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 0.0 , 32.0 , 227.0 , 252.0 , 232.0 , 0.0 , 0.0 , 0

# 통계

In [66]:
from pyspark.mllib.stat import Statistics

parallelData = spark.sparkContext.parallelize([1.0, 2.0, 5.0, 4.0, 3.0, 3.3, 5.5])

testResult = Statistics.kolmogorovSmirnovTest(parallelData, "norm")
print(testResult)

Kolmogorov-Smirnov test summary:
degrees of freedom = 0 
statistic = 0.841344746068543 
pValue = 5.06089025353873E-6 
Very strong presumption against null hypothesis: Sample follows theoretical distribution.


In [69]:
df = spark.range(0,10)
df.show()
df.select('id')
from pyspark.sql.functions import rand, randn
colUniform = rand(seed=10).alias("uniform")
colNormal=randn(seed=27).alias("normal")
df3=df.select("id", colUniform,colNormal)
df3.show()
df3.describe().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.41371264720975787|  0.5888539012978773|
|  1| 0.1982919638208397| 0.06157382353970104|
|  2|0.12030715258495939|  1.0854146699817222|
|  3|0.44292918521277047| -0.4798519469521663|
|  4| 0.8898784253886249| -0.8820294772950535|
|  5| 0.2731073068483362|-0.15116027592854422|
|  6|   0.87079354700073|-0.27674189870783683|
|  7|0.27149331793166864|-0.18575112254167045|
|  8| 0.6037143578435027|   0.734722467897308|
|  9| 0.1435668838975337|-0.30123700668427145|
+---+-------------------+--------------------+

+-------+------------------+-------------------+--------------------+
|summary|                id|            uniform|              normal|
+-------+------------------+-------------------+--------------------+
|  count|                10|                 1

In [70]:
df=spark.createDataFrame([(1,2,3) if i%2==0 else (i,2*i,i%4) for i in range(100)],["a","b","c"])
df.show(10)
freq = df.stat.freqItems(["a","b","c"],0.6)
print freq.show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  1|  2|  1|
|  1|  2|  3|
|  3|  6|  3|
|  1|  2|  3|
|  5| 10|  1|
|  1|  2|  3|
|  7| 14|  3|
|  1|  2|  3|
|  9| 18|  1|
+---+---+---+
only showing top 10 rows

+-----------+-----------+-----------+
|a_freqItems|b_freqItems|c_freqItems|
+-----------+-----------+-----------+
|        [1]|        [2]|        [3]|
+-----------+-----------+-----------+

None


# DataFrame 변환

### 단어 빈도 계산하기 (PYTHON)

In [91]:
doc=[
    "When I find myself in times of trouble",
    "Mother Mary comes to me",
    "Speaking words of wisdom, let it be",
    "And in my hour of darkness",
    "She is standing right in front of me",
    "Speaking words of wisdom, let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Whisper words of wisdom, let it be"
]
d={}
for sentence in doc:
    words=sentence.split()
    for word in words:
        if word in d:
            d[word]+=1
        else:
            d[word]=1
            
for k,v in d.iteritems():
    print k,v

right 1
be 7
is 1
When 1
it 7
in 3
Mary 1
Speaking 2
standing 1
darkness 1
find 1
wisdom, 3
to 1
Let 4
And 1
I 1
let 3
She 1
words 3
Mother 1
front 1
trouble 1
me 2
myself 1
hour 1
of 6
times 1
Whisper 1
my 1
comes 1
